In [1]:
import os
import sys
os.environ['SPARK_HOME']='/home/cloudera/spark230hadoop26'
os.environ['PYLIB']=os.environ['SPARK_HOME']+'/python/lib'
sys.path.insert(0,os.environ['PYLIB']+'/py4j-0.10.6-src.zip')
sys.path.insert(1,os.environ['PYLIB']+'/pyspark.zip')

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np

In [3]:
spark = SparkSession.builder.appName('JSONSparkSQL') \
.config('hive.metastore.uris', 'thrift://localhost:9083') \
.enableHiveSupport().getOrCreate()

In [4]:
# Having set the driver and driver options we should have spark representing spark session 
# available straight away
spark.version

'2.3.0'

In [5]:
sc = spark.sparkContext

In [6]:
from pyspark.sql.types import *
json_schema = StructType(
[StructField('battery_level', LongType()), StructField('c02_level', LongType()), 
StructField('cca3', StringType()), StructField('cn', StringType()),
StructField('device_id', LongType()), StructField('device_type', StringType()), 
StructField('signal', LongType()), StructField('ip', StringType()) , 
StructField('temp', LongType()) , StructField('timestamp', TimestampType()) ])

In [7]:
eventsData = [
 (0, """{"device_id": 0, "device_type": "sensor-ipad", "ip": "68.161.225.1", "cca3": "USA", "cn": "United States", "temp": 25, "signal": 23, "battery_level": 8, "c02_level": 917, "timestamp" :1475600496 }"""),
 (1, """{"device_id": 1, "device_type": "sensor-igauge", "ip": "213.161.254.1", "cca3": "NOR", "cn": "Norway", "temp": 30, "signal": 18, "battery_level": 6, "c02_level": 1413, "timestamp" :1475600498 }"""),
 (2, """{"device_id": 2, "device_type": "sensor-ipad", "ip": "88.36.5.1", "cca3": "ITA", "cn": "Italy", "temp": 18, "signal": 25, "battery_level": 5, "c02_level": 1372, "timestamp" :1475600500 }"""),
 (3, """{"device_id": 3, "device_type": "sensor-inest", "ip": "66.39.173.154", "cca3": "USA", "cn": "United States", "temp": 47, "signal": 12, "battery_level": 1, "c02_level": 1447, "timestamp" :1475600502 }"""),
(4, """{"device_id": 4, "device_type": "sensor-ipad", "ip": "203.82.41.9", "cca3": "PHL", "cn": "Philippines", "temp": 29, "signal": 11, "battery_level": 0, "c02_level": 983, "timestamp" :1475600504 }"""),
(5, """{"device_id": 5, "device_type": "sensor-istick", "ip": "204.116.105.67", "cca3": "USA", "cn": "United States", "temp": 50, "signal": 16, "battery_level": 8, "c02_level": 1574, "timestamp" :1475600506 }"""),
(6, """{"device_id": 6, "device_type": "sensor-ipad", "ip": "220.173.179.1", "cca3": "CHN", "cn": "China", "temp": 21, "signal": 18, "battery_level": 9, "c02_level": 1249, "timestamp" :1475600508 }"""),
(7, """{"device_id": 7, "device_type": "sensor-ipad", "ip": "118.23.68.227", "cca3": "JPN", "cn": "Japan", "temp": 27, "signal": 15, "battery_level": 0, "c02_level": 1531, "timestamp" :1475600512 }"""),
(8 ,""" {"device_id": 8, "device_type": "sensor-inest", "ip": "208.109.163.218", "cca3": "USA", "cn": "United States", "temp": 40, "signal": 16, "battery_level": 9, "c02_level": 1208, "timestamp" :1475600514 }"""),
(9,"""{"device_id": 9, "device_type": "sensor-ipad", "ip": "88.213.191.34", "cca3": "ITA", "cn": "Italy", "temp": 19, "signal": 11, "battery_level": 0, "c02_level": 1171, "timestamp" :1475600516 }"""),
(10,"""{"device_id": 10, "device_type": "sensor-igauge", "ip": "68.28.91.22", "cca3": "USA", "cn": "United States", "temp": 32, "signal": 26, "battery_level": 7, "c02_level": 886, "timestamp" :1475600518 }"""),
(11,"""{"device_id": 11, "device_type": "sensor-ipad", "ip": "59.144.114.250", "cca3": "IND", "cn": "India", "temp": 46, "signal": 25, "battery_level": 4, "c02_level": 863, "timestamp" :1475600520 }"""),
(12, """{"device_id": 12, "device_type": "sensor-igauge", "ip": "193.156.90.200", "cca3": "NOR", "cn": "Norway", "temp": 18, "signal": 26, "battery_level": 8, "c02_level": 1220, "timestamp" :1475600522 }"""),
(13, """{"device_id": 13, "device_type": "sensor-ipad", "ip": "67.185.72.1", "cca3": "USA", "cn": "United States", "temp": 34, "signal": 20, "battery_level": 8, "c02_level": 1504, "timestamp" :1475600524 }"""),
(14, """{"device_id": 14, "device_type": "sensor-inest", "ip": "68.85.85.106", "cca3": "USA", "cn": "United States", "temp": 39, "signal": 17, "battery_level": 8, "c02_level": 831, "timestamp" :1475600526 }"""),
(15, """{"device_id": 15, "device_type": "sensor-ipad", "ip": "161.188.212.254", "cca3": "USA", "cn": "United States", "temp": 27, "signal": 26, "battery_level": 5, "c02_level": 1378, "timestamp" :1475600528 }"""),
(16, """{"device_id": 16, "device_type": "sensor-igauge", "ip": "221.3.128.242", "cca3": "CHN", "cn": "China", "temp": 10, "signal": 24, "battery_level": 6, "c02_level": 1423, "timestamp" :1475600530 }"""),
(17, """{"device_id": 17, "device_type": "sensor-ipad", "ip": "64.124.180.215", "cca3": "USA", "cn": "United States", "temp": 38, "signal": 17, "battery_level": 9, "c02_level": 1304, "timestamp" :1475600532 }"""),
(18, """{"device_id": 18, "device_type": "sensor-igauge", "ip": "66.153.162.66", "cca3": "USA", "cn": "United States", "temp": 26, "signal": 10, "battery_level": 0, "c02_level": 902, "timestamp" :1475600534 }"""),
(19, """{"device_id": 19, "device_type": "sensor-ipad", "ip": "193.200.142.254", "cca3": "AUT", "cn": "Austria", "temp": 32, "signal": 27, "battery_level": 5, "c02_level": 1282, "timestamp" :1475600536 }""")]

In [8]:
eventsFromJSONDF = sc.parallelize(eventsData).toDF(['id', 'json'])

In [9]:
eventsFromJSONDF.printSchema()

root
 |-- id: long (nullable = true)
 |-- json: string (nullable = true)



In [10]:
from pyspark.sql.functions import *

In [11]:
eventsFromJSONDF.select(get_json_object('json', '$.device_type').alias('dev_type'),
                       get_json_object('json', '$.ip').alias('ip'),
                       get_json_object('json', '$.temp').alias('temp')).show()

+-------------+---------------+----+
|     dev_type|             ip|temp|
+-------------+---------------+----+
|  sensor-ipad|   68.161.225.1|  25|
|sensor-igauge|  213.161.254.1|  30|
|  sensor-ipad|      88.36.5.1|  18|
| sensor-inest|  66.39.173.154|  47|
|  sensor-ipad|    203.82.41.9|  29|
|sensor-istick| 204.116.105.67|  50|
|  sensor-ipad|  220.173.179.1|  21|
|  sensor-ipad|  118.23.68.227|  27|
| sensor-inest|208.109.163.218|  40|
|  sensor-ipad|  88.213.191.34|  19|
|sensor-igauge|    68.28.91.22|  32|
|  sensor-ipad| 59.144.114.250|  46|
|sensor-igauge| 193.156.90.200|  18|
|  sensor-ipad|    67.185.72.1|  34|
| sensor-inest|   68.85.85.106|  39|
|  sensor-ipad|161.188.212.254|  27|
|sensor-igauge|  221.3.128.242|  10|
|  sensor-ipad| 64.124.180.215|  38|
|sensor-igauge|  66.153.162.66|  26|
|  sensor-ipad|193.200.142.254|  32|
+-------------+---------------+----+



In [14]:
eventsDF = eventsFromJSONDF.select(from_json(eventsFromJSONDF.json, json_schema).alias('device'))

In [15]:
eventsDF.printSchema()

root
 |-- device: struct (nullable = true)
 |    |-- battery_level: long (nullable = true)
 |    |-- c02_level: long (nullable = true)
 |    |-- cca3: string (nullable = true)
 |    |-- cn: string (nullable = true)
 |    |-- device_id: long (nullable = true)
 |    |-- device_type: string (nullable = true)
 |    |-- signal: long (nullable = true)
 |    |-- ip: string (nullable = true)
 |    |-- temp: long (nullable = true)
 |    |-- timestamp: timestamp (nullable = true)



In [18]:
eventsDF.select('device.*').show()

+-------------+---------+----+-------------+---------+-------------+------+---------------+----+-------------------+
|battery_level|c02_level|cca3|           cn|device_id|  device_type|signal|             ip|temp|          timestamp|
+-------------+---------+----+-------------+---------+-------------+------+---------------+----+-------------------+
|            8|      917| USA|United States|        0|  sensor-ipad|    23|   68.161.225.1|  25|2016-10-04 22:31:36|
|            6|     1413| NOR|       Norway|        1|sensor-igauge|    18|  213.161.254.1|  30|2016-10-04 22:31:38|
|            5|     1372| ITA|        Italy|        2|  sensor-ipad|    25|      88.36.5.1|  18|2016-10-04 22:31:40|
|            1|     1447| USA|United States|        3| sensor-inest|    12|  66.39.173.154|  47|2016-10-04 22:31:42|
|            0|      983| PHL|  Philippines|        4|  sensor-ipad|    11|    203.82.41.9|  29|2016-10-04 22:31:44|
|            8|     1574| USA|United States|        5|sensor-ist

In [19]:
eventsDF.select('device.ip', 'device.cn', 'device.temp').show()

+---------------+-------------+----+
|             ip|           cn|temp|
+---------------+-------------+----+
|   68.161.225.1|United States|  25|
|  213.161.254.1|       Norway|  30|
|      88.36.5.1|        Italy|  18|
|  66.39.173.154|United States|  47|
|    203.82.41.9|  Philippines|  29|
| 204.116.105.67|United States|  50|
|  220.173.179.1|        China|  21|
|  118.23.68.227|        Japan|  27|
|208.109.163.218|United States|  40|
|  88.213.191.34|        Italy|  19|
|    68.28.91.22|United States|  32|
| 59.144.114.250|        India|  46|
| 193.156.90.200|       Norway|  18|
|    67.185.72.1|United States|  34|
|   68.85.85.106|United States|  39|
|161.188.212.254|United States|  27|
|  221.3.128.242|        China|  10|
| 64.124.180.215|United States|  38|
|  66.153.162.66|United States|  26|
|193.200.142.254|      Austria|  32|
+---------------+-------------+----+



In [20]:
eventsDF.select('device.ip', 'device.cn', 'device.temp') \
.filter('temp < 20').show()

+--------------+------+----+
|            ip|    cn|temp|
+--------------+------+----+
|     88.36.5.1| Italy|  18|
| 88.213.191.34| Italy|  19|
|193.156.90.200|Norway|  18|
| 221.3.128.242| China|  10|
+--------------+------+----+



In [21]:
eventsDF.select(to_json(struct('device.ip', 'device.cn', 'device.temp')))\
.toDF('devices').show(truncate=False)

+-------------------------------------------------------+
|devices                                                |
+-------------------------------------------------------+
|{"ip":"68.161.225.1","cn":"United States","temp":25}   |
|{"ip":"213.161.254.1","cn":"Norway","temp":30}         |
|{"ip":"88.36.5.1","cn":"Italy","temp":18}              |
|{"ip":"66.39.173.154","cn":"United States","temp":47}  |
|{"ip":"203.82.41.9","cn":"Philippines","temp":29}      |
|{"ip":"204.116.105.67","cn":"United States","temp":50} |
|{"ip":"220.173.179.1","cn":"China","temp":21}          |
|{"ip":"118.23.68.227","cn":"Japan","temp":27}          |
|{"ip":"208.109.163.218","cn":"United States","temp":40}|
|{"ip":"88.213.191.34","cn":"Italy","temp":19}          |
|{"ip":"68.28.91.22","cn":"United States","temp":32}    |
|{"ip":"59.144.114.250","cn":"India","temp":46}         |
|{"ip":"193.156.90.200","cn":"Norway","temp":18}        |
|{"ip":"67.185.72.1","cn":"United States","temp":34}    |
|{"ip":"68.85.

In [23]:
eventsDF.select(to_json(struct('device.ip', 'device.cn', 'device.temp')))\
.toDF('devices').printSchema()

root
 |-- devices: string (nullable = true)



In [24]:
eventsDF.select(to_json(struct('*')))\
.toDF('devices').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|devices                                                                                                                                                                                                              |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{"device":{"battery_level":8,"c02_level":917,"cca3":"USA","cn":"United States","device_id":0,"device_type":"sensor-ipad","signal":23,"ip":"68.161.225.1","temp":25,"timestamp":"2016-10-04T22:31:36.000+05:30"}}     |
|{"device":{"battery_level":6,"c02_level":1413,"cca3":"NOR","cn":"Norway","device_id":1,"device_type":"sensor-igauge","signal":18,"ip":"

In [18]:
dataDS = ["""
{
"dc_id": "dc-101",
"source": {
    "sensor-igauge": {
      "id": 10,
      "ip": "68.28.91.22",
      "description": "Sensor attached to the container ceilings",
      "temp":35,
      "c02_level": 1475,
      "geo": {"lat":38.00, "long":97.00}                        
    },
    "sensor-ipad": {
      "id": 13,
      "ip": "67.185.72.1",
      "description": "Sensor ipad attached to carbon cylinders",
      "temp": 34,
      "c02_level": 1370,
      "geo": {"lat":47.41, "long":-122.00}
    },
    "sensor-inest": {
      "id": 8,
      "ip": "208.109.163.218",
      "description": "Sensor attached to the factory ceilings",
      "temp": 40,
      "c02_level": 1346,
      "geo": {"lat":33.61, "long":-111.89}
    },
    "sensor-istick": {
      "id": 5,
      "ip": "204.116.105.67",
      "description": "Sensor embedded in exhaust pipes in the ceilings",
      "temp": 40,
      "c02_level": 1574,
      "geo": {"lat":35.93, "long":-85.46}
    }
  }
}"""]

In [19]:
dataDS = ["""{ "dc_id": "dc-101", "source": { "sensor-igauge": { "id": 10, "ip": "68.28.91.22", "description": "Sensor attached to the container ceilings", "temp":35, "c02_level": 1475, "geo": {"lat":38.00, "long":97.00} }, "sensor-ipad": { "id": 13, "ip": "67.185.72.1", "description": "Sensor ipad attached to carbon cylinders", "temp": 34, "c02_level": 1370, "geo": {"lat":47.41, "long":-122.00} }, "sensor-inest": { "id": 8, "ip": "208.109.163.218", "description": "Sensor attached to the factory ceilings", "temp": 40, "c02_level": 1346, "geo": {"lat":33.61, "long":-111.89} }, "sensor-istick": { "id": 5, "ip": "204.116.105.67", "description": "Sensor embedded in exhaust pipes in the ceilings", "temp": 40, "c02_level": 1574, "geo": {"lat":35.93, "long":-85.46} } } }"""]

In [20]:
len(dataDS)

1

In [21]:
from pyspark.sql import Row
cmplxDF = sc.parallelize(dataDS).map(lambda x: Row(x)).toDF(['nstr'])

In [22]:
cmplxDF.printSchema()

root
 |-- nstr: string (nullable = true)



In [23]:
cmplxDF.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|nstr                                                                                                                                                                                                                                       

In [24]:
schema = StructType([StructField("dc_id", StringType()),
                    StructField("source", 
                                MapType(StringType(), 
                                StructType([
                                   StructField("description", StringType()),
                                   StructField("ip", StringType()),
                                   StructField("id", LongType()),
                                   StructField("temp", LongType()),
                                   StructField("c02_level", LongType()),
                                   StructField("geo", 
                                               StructType([
                                                   StructField("lat", DoubleType()),
                                                   StructField("long", DoubleType())
                                               ]
                                               )
                                              )
                                ])
                                       )
                               )
                    ])

In [25]:
cmplxDF.rdd.collect()

[Row(nstr='{ "dc_id": "dc-101", "source": { "sensor-igauge": { "id": 10, "ip": "68.28.91.22", "description": "Sensor attached to the container ceilings", "temp":35, "c02_level": 1475, "geo": {"lat":38.00, "long":97.00} }, "sensor-ipad": { "id": 13, "ip": "67.185.72.1", "description": "Sensor ipad attached to carbon cylinders", "temp": 34, "c02_level": 1370, "geo": {"lat":47.41, "long":-122.00} }, "sensor-inest": { "id": 8, "ip": "208.109.163.218", "description": "Sensor attached to the factory ceilings", "temp": 40, "c02_level": 1346, "geo": {"lat":33.61, "long":-111.89} }, "sensor-istick": { "id": 5, "ip": "204.116.105.67", "description": "Sensor embedded in exhaust pipes in the ceilings", "temp": 40, "c02_level": 1574, "geo": {"lat":35.93, "long":-85.46} } } }')]

In [26]:
df = spark.read.schema(schema).json(cmplxDF.rdd)

In [27]:
df.printSchema()

root
 |-- dc_id: string (nullable = true)
 |-- source: map (nullable = true)
 |    |-- key: string
 |    |-- value: struct (valueContainsNull = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- temp: long (nullable = true)
 |    |    |-- c02_level: long (nullable = true)
 |    |    |-- geo: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- long: double (nullable = true)



In [28]:
df.select(explode('source')).show()
df.count()

+---+-----+
|key|value|
+---+-----+
+---+-----+



1

In [29]:
spark.read.parquet('hdfs://localhost:8020/user/cloudera/stdf_parquet').show()

+------+----+----+------+---------+
|symbol|  yr|mnth|   qty|      vlu|
+------+----+----+------+---------+
|  INFY|2017|   5|  2000|2164550.0|
|  INFY|2017|   5|  1954|2174352.0|
|  INFY|2017|   6|  2341|2934231.0|
|  INFY|2017|   6|  1814|1904557.0|
|  SBIN|2017|   5|200061|3164550.0|
|  SBIN|2017|   5|211954|3174352.0|
|  SBIN|2017|   6|222341|3434234.0|
|  SBIN|2017|   6|301814|4590455.0|
+------+----+----+------+---------+



In [35]:
spark.read.json(schema=schema,path='file:///home/cloudera/cmplx.json').show()

+-----+------+
|dc_id|source|
+-----+------+
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
+-----+------+
only showing top 20 rows



In [32]:
df.printSchema()

root
 |-- dc_id: string (nullable = true)
 |-- source: map (nullable = true)
 |    |-- key: string
 |    |-- value: struct (valueContainsNull = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- temp: long (nullable = true)
 |    |    |-- c02_level: long (nullable = true)
 |    |    |-- geo: struct (nullable = true)
 |    |    |    |-- lat: double (nullable = true)
 |    |    |    |-- long: double (nullable = true)



In [33]:
df.show()

+-----+------+
|dc_id|source|
+-----+------+
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
| null|  null|
+-----+------+
only showing top 20 rows

